In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob

# --------------------------------------------------------------
# Read single CSV file
# --------------------------------------------------------------

In [2]:
single_file_acc=pd.read_csv("../../data/raw/MetaMotion/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
#print(single_file_acc.head())

single_file_gyr=pd.read_csv("../../data/raw/MetaMotion/MetaMotion/A-bench-heavy2_MetaWear_2019-01-14T14.27.00.784_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
#print(single_file_gyr.head())

In [3]:
# --------------------------------------------------------------
# List all data in data/raw/MetaMotion
# --------------------------------------------------------------
files=glob("../../data/raw/MetaMotion/MetaMotion/*.csv")
#print(len(files))

# --------------------------------------------------------------
# Extract features from filename
# --------------------------------------------------------------

In [4]:
data_path="../../data/raw/MetaMotion/MetaMotion"
f=files[1]
print(f)

participant=f.split("-")[0].replace(data_path,"")
label=f.split("-")[1]
category=f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

df=pd.read_csv(f)
df["participant"]=participant
df["label"]=label
df["category"]=category
#print(df)

../../data/raw/MetaMotion/MetaMotion\A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv


# --------------------------------------------------------------
# Read all files
# --------------------------------------------------------------

In [5]:
acc_df=pd.DataFrame()
gyr_df=pd.DataFrame()

acc_set=1
gyr_set=1

for f in files:
    participant=f.split("-")[0].replace(data_path,"")
    label=f.split("-")[1]
    category=f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")
    #print(category)

    df=pd.read_csv(f)
    df["participant"]=participant
    df["label"]=label
    df["category"]=category

    if "Accelerometer" in f:
        df["set"]=acc_set
        acc_set+=1
        acc_df=pd.concat([acc_df,df])
    if "Gyroscope" in f:
        df["set"]=gyr_set
        gyr_set+=1
        gyr_df=pd.concat([gyr_df,df])

#print(gyr_df.shape)


In [6]:
acc_df.info()
pd.to_datetime(df["epoch (ms)"],unit="ms")
acc_df.index=pd.to_datetime(acc_df["epoch (ms)"],unit="ms")
gyr_df.index=pd.to_datetime(gyr_df["epoch (ms)"],unit="ms")

del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]



<class 'pandas.core.frame.DataFrame'>
Index: 23578 entries, 0 to 241
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   epoch (ms)    23578 non-null  int64  
 1   time (01:00)  23578 non-null  object 
 2   elapsed (s)   23578 non-null  float64
 3   x-axis (g)    23578 non-null  float64
 4   y-axis (g)    23578 non-null  float64
 5   z-axis (g)    23578 non-null  float64
 6   participant   23578 non-null  object 
 7   label         23578 non-null  object 
 8   category      23578 non-null  object 
 9   set           23578 non-null  int64  
dtypes: float64(4), int64(2), object(4)
memory usage: 2.0+ MB


In [7]:
# --------------------------------------------------------------
# Turn into function
# --------------------------------------------------------------
files=glob("../../data/raw/MetaMotion/MetaMotion/*.csv")

def read_data_from_files(files):
    acc_df=pd.DataFrame()
    gyr_df=pd.DataFrame()

    acc_set=1
    gyr_set=1

    for f in files:
        participant=f.split("-")[0].replace(data_path,"")
        label=f.split("-")[1]
        category=f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")
        #print(category)

        df=pd.read_csv(f)
        df["participant"]=participant
        df["label"]=label
        df["category"]=category

        if "Accelerometer" in f:
            df["set"]=acc_set
            acc_set+=1
            acc_df=pd.concat([acc_df,df])
        if "Gyroscope" in f:
            df["set"]=gyr_set
            gyr_set+=1
            gyr_df=pd.concat([gyr_df,df])
    acc_df.index=pd.to_datetime(acc_df["epoch (ms)"],unit="ms")
    gyr_df.index=pd.to_datetime(gyr_df["epoch (ms)"],unit="ms")

    del acc_df["epoch (ms)"]
    del acc_df["time (01:00)"]
    del acc_df["elapsed (s)"]

    del gyr_df["epoch (ms)"]
    del gyr_df["time (01:00)"]
    del gyr_df["elapsed (s)"]

    return acc_df,gyr_df

acc_df,gyr_df=read_data_from_files(files)

In [8]:
# --------------------------------------------------------------
# Merging datasets
# --------------------------------------------------------------
data_merged=pd.concat([acc_df.iloc[:,:3],gyr_df],axis=1)
# rename columns
data_merged.columns=["acc_x","acc_y","acc_z","gyr_x","gyr_y","gyr_z","participant","label","category","set"]


In [9]:
# --------------------------------------------------------------
# Resample data (frequency conversion)
# --------------------------------------------------------------

# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz

sampling={"acc_x":"mean","acc_y":"mean","acc_z":"mean","gyr_x":"mean","gyr_y":"mean","gyr_z":"mean","participant":"last","label":"last","category":"last","set":"last",}
data_merged[:1000].resample(rule="200ms").apply(sampling)
# split by day
days=[g for n,g in data_merged.groupby(pd.Grouper(freq="D"))]
data_resampled=pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])
data_resampled["set"]= data_resampled["set"].astype("int")
data_resampled.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   int32  
dtypes: float64(6), int32(1), object(3)
memory usage: 739.0+ KB


In [10]:
# --------------------------------------------------------------
# Export dataset
# --------------------------------------------------------------
data_resampled.to_pickle("../../data/interim/01_data_processed.pkl")

In [11]:
pip install matplotlib

  Using cached contourpy-1.1.1-cp38-cp38-win_amd64.whl.metadata (5.9 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached kiwisolver-1.4.7-cp38-cp38-win_amd64.whl.metadata (6.4 kB)
  Using cached pillow-10.4.0-cp38-cp38-win_amd64.whl.metadata (9.3 kB)
  Using cached pyparsing-3.1.4-py3-none-any.whl.metadata (5.1 kB)
   ---------------------------------------- 0.0/7.5 MB ? eta -:--:--
   -- ------------------------------------- 0.5/7.5 MB 3.4 MB/s eta 0:00:03
   ----- ---------------------------------- 1.0/7.5 MB 2.5 MB/s eta 0:00:03
   -------- ------------------------------- 1.6/7.5 MB 2.7 MB/s eta 0:00:03
   ------------ --------------------------- 2.4/7.5 MB 3.1 MB/s eta 0:00:02
   --------------- ------------------------ 2.9/7.5 MB 2.8 MB/s eta 0:00:02
   ------------------ --------------------- 3.4/7.5 MB 2.7 MB/s eta 0:00:02
   ----------------------- ---------------- 4.5/7.5 MB 3.0 MB/s eta 0:00:02
   ----------------------------- ---------- 5.5/7.5

In [13]:
pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
output_dir = "../../reports/figures/"
print(os.makedirs(output_dir, exist_ok=True))


None


In [5]:
output_dir="../../reports/figures/"
if os.path.exists(output_dir):
    print("Yes")
else:
    print("NO")

Yes


In [7]:
print(os.path.abspath(output_dir))

c:\Users\manis\Desktop\Fitness Tracker\reports\figures


In [13]:
import os

save_dir = "../../reports/figures/"
os.makedirs(save_dir, exist_ok=True)  # Ensure directory exists
plt.savefig(os.path.join(save_dir, f"{label.title()} ({participant}).png"))


In [2]:
pip install scikit-learn

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.3 MB 3.3 MB/s eta 0:00:03
   ------- -------------------------------- 1.8/9.3 MB 4.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.4/9.3 MB 3.8 MB/s eta 0:00:02
   -------------- ------------------------- 3.4/9.3 MB 4.0 MB/s eta 0:00:02
   ------------------- -------------------- 4.5/9.3 MB 4.2 MB/s eta 0:00:02
   --------------------- ------------------ 5.0/9.3 MB 3.9 MB/s eta 0:00:02
   ------------------------ --------------- 5.8/9.3 MB 3.9 MB/s eta 0:00:01
   ---------------------------- ----------- 6.6/9.3 MB 4.0 MB/s eta 0:00:01
   ------------------------------- -------- 7.3/9.3 MB 3.9 MB/s eta 0:00:01
   ------------------------------------- -- 8.

In [2]:
pip install DataTransformation

Note: you may need to restart the kernel to use updated packages.


ERROR: Ignored the following versions that require a different python version: 0.1.0 Requires-Python <4.0,>=3.9; 0.1.1 Requires-Python <4.0,>=3.9
ERROR: Could not find a version that satisfies the requirement DataTransformation (from versions: none)
ERROR: No matching distribution found for DataTransformation


In [1]:
pip install TemporalAbstraction

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement TemporalAbstraction (from versions: none)
ERROR: No matching distribution found for TemporalAbstraction


In [10]:
data_resampled["participant"]

epoch (ms)
2019-01-11 15:08:05.200    \B
2019-01-11 15:08:05.400    \B
2019-01-11 15:08:05.600    \B
2019-01-11 15:08:05.800    \B
2019-01-11 15:08:06.000    \B
                           ..
2019-01-20 17:33:27.000    \E
2019-01-20 17:33:27.200    \E
2019-01-20 17:33:27.400    \E
2019-01-20 17:33:27.600    \E
2019-01-20 17:33:27.800    \E
Name: participant, Length: 9009, dtype: object

In [11]:
data_resampled['participant']=df['participant'].str.replace(r'^\\','"',regex=True)

In [12]:
data_resampled["participant"]

epoch (ms)
2019-01-11 15:08:05.200    NaN
2019-01-11 15:08:05.400    NaN
2019-01-11 15:08:05.600    NaN
2019-01-11 15:08:05.800    NaN
2019-01-11 15:08:06.000    NaN
                          ... 
2019-01-20 17:33:27.000    NaN
2019-01-20 17:33:27.200    NaN
2019-01-20 17:33:27.400    NaN
2019-01-20 17:33:27.600    NaN
2019-01-20 17:33:27.800    NaN
Name: participant, Length: 9009, dtype: object

In [13]:
data_resampled

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,NaN,bench,heavy,30
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,NaN,bench,heavy,30
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,NaN,bench,heavy,30
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,NaN,bench,heavy,30
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,NaN,bench,heavy,30
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,NaN,row,medium,90
2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,NaN,row,medium,90
2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,NaN,row,medium,90
